In [157]:
# imports - - - - - - - - -
import numpy 
import torch
import csv
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

# Get Table Data - - - - - - - - -

path = (r"C:\Users\sotiv\data\p1ch4\winequality-white.csv")
wine_as_numpy = numpy.loadtxt(
    path, dtype = numpy.float32, delimiter = ";", skiprows=1
) #Puts tabular data in CSV into an array 

wine_as_numpy #Tabular data in NumPy array!

# Table Data Documentation - - - - - - - - -

readable_format = next(csv.reader(open(path), delimiter = ";"))

wine_as_numpy.shape, readable_format

wine_as_torch = torch.from_numpy(wine_as_numpy)

wine_as_torch, wine_as_torch.shape #Get the wine data as a torch tensor!

#  Representing Scores and Data Organizaiton - - - - - - - - - - - - - - - - - -

wine_data = wine_as_torch[:, :-1] #Keep data excluding the scores in last column

score_tensor = wine_as_torch[:, -1] #Make score data it's own tensor

score_tensor_int = score_tensor.long() #Change score data into integers 

score_tensor_int

onehot_scoretensor = torch.zeros(score_tensor.shape[0], 10) #shape(0) makes it an int 4629

onehot_indices = score_tensor_int.unsqueeze(1) #onehot needs a column of values, so unsqueeze as necessary

onehot_scoretensor.scatter_(1, score_tensor_int.unsqueeze(1), 1.0) # 

onehot_scoretensor

# - - - - - - - - - - - - - - - - - -

winedata_mean = torch.mean(wine_data, dim = 0) #Finds mean of each column in wine_data. 
#* Why is it important to specify the dimension being 0?

winedata_var = torch.var(wine_data, dim = 0)

winedata_normalized = (wine_data - winedata_mean) / (torch.sqrt(winedata_var))

winedata_normalized 

# Getting Significant Indices- - - - - - - - - - - - - - - - - - - - - - - - - - -

#Here we get the indices of data with certain characteristics
bad_wine = (score_tensor_int <= 3)
ok_wine = (score_tensor_int > 3) & (score_tensor_int < 7)
scrumptious_wine = (score_tensor_int > 7)

#Here I can find how many indices meet that characteristic
bad_wine.sum()

#Here I can find those specific values associated with that characteristic

bad_data = wine_data[bad_wine] #returns rows & columns from data that have the quality that stinky_wine has.
ok_data = wine_data[ok_wine]
scrumptious_data = wine_data[scrumptious_wine]

bad_mean = torch.mean(bad_data, dim = 0) #Means of those rows & columns
ok_mean = torch.mean(ok_data, dim = 0)
scrumptious_mean = torch.mean(scrumptious_data, dim = 0)


#For bad, ok, and good wine, here are the characteristics they have!

for i, args in enumerate(zip(readable_format, bad_mean, ok_mean, scrumptious_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args)) 

# - - - - - - - - - - - - - - - - - - - - - - - - - - -

#Given the data, we estimate a certain threshold for a characteristic that would define 'Good wine'
sulfer_threshold = 141.83 


sulfer_data = wine_data[:, 6]

#Based on our threshold, we make a boolean tensor

predicted_good_wines = torch.lt(sulfer_data, sulfer_threshold) #Creates true/false tensor depending on if arg1 is LESS THAN (lt) arg2

acc_good_wines = (score_tensor_int > 5)

#Here, we compare the amount of good wines based on our predicted threshold, and the actual scores. It's kinda far off,
#which goes to show that relying on one feature is a bad judge of character!! :D

predicted_good_wines.sum(), acc_good_wines.sum()

matches = torch.sum(predicted_good_wines & acc_good_wines).item()
predicted = torch.sum(predicted_good_wines).item()
actual = torch.sum(acc_good_wines).item()

matches






 0 fixed acidity          7.60   6.89   6.68
 1 volatile acidity       0.33   0.28   0.28
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.63
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  36.63
 6 total sulfur dioxide 170.60 141.83 125.88
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.49
10 alcohol               10.34  10.26  11.65


2018